In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
data = pd.read_csv('burglaries_preprocessed.csv')


In [3]:

def create_features(df):

    df = df.set_index('Month')
    df.index = pd.to_datetime(df.index)
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    
    return df

data = create_features(data)

In [7]:
data

,Unnamed: 0,Crime ID,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,Ward,quarter,month,year
Month,,,,,,,,,,,,,,,,
2010-12-01,0,NaN,Metropolitan Police Service,Metropolitan Police Service,-0.201877,51.655538,On or near High Street,E01000248,Barnet 001A,Burglary,NaN,NaN,High Barnet,4,12,2010
2010-12-01,1,NaN,Metropolitan Police Service,Metropolitan Police Service,-0.207853,51.654317,On or near The Avenue,E01000248,Barnet 001A,Burglary,NaN,NaN,High Barnet,4,12,2010
2010-12-01,2,NaN,Metropolitan Police Service,Metropolitan Police Service,-0.202510,51.656348,On or near Bruce Road,E01000248,Barnet 001A,Burglary,NaN,NaN,High Barnet,4,12,2010
2010-12-01,3,NaN,Metropolitan Police Service,Metropolitan Police Service,-0.206779,51.654768,On or near The Drive,E01000248,Barnet 001A,Burglary,NaN,NaN,High Barnet,4,12,2010
2010-12-01,4,NaN,Metropolitan Police Service,Metropolitan Police Service,-0.209537,51.655223,On or near Marriott Road,E01000249,Barnet 001B,Burglary,NaN,NaN,High Barnet,4,12,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-01,44133,fda0870be993d2c9352ae14a540bf529c15a7aa9d92a4e...,Metropolitan Police Service,Metropolitan Police Service,-0.202163,51.559100,On or near Hocroft Road,E01000139,Barnet 041B,Burglary,Under investigation,NaN,Childs Hill,1,3,2023
2023-03-01,44134,c6c2d9a772695865f4ca33fce59c26c48fe3a1cdac8c22...,Metropolitan Police Service,Metropolitan Police Service,-0.202163,51.559100,On or near Hocroft Road,E01000139,Barnet 041B,Burglary,Under investigation,NaN,Childs Hill,1,3,2023
2023-03-01,44135,e5bab2bd4f31de44515dbf125b85284392d0349139a1d3...,Metropolitan Police Service,Metropolitan Police Service,-0.197843,51.561093,On or near Church Walk,E01000140,Barnet 041C,Burglary,Under investigation,NaN,Childs Hill,1,3,2023


In [9]:
train = data.loc[data.index < '2022-01-01']
# val = data.loc['2022-01-01' <= data.index < '2022-07-01' ]
test = data.loc[data.index >= '2022-01-01']

In [10]:
test.index.unique()

DatetimeIndex(['2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01',
               '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01',
               '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01',
               '2023-01-01', '2023-02-01', '2023-03-01'],
              dtype='datetime64[ns]', name='Month', freq=None)